<a href="https://colab.research.google.com/github/fjadidi2001/fake_news_detection/blob/main/BertGnn_Apr1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The BERT model was proposed in BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding by Jacob Devlin, Ming-Wei Chang, Kenton Lee and Kristina Toutanova. It’s a bidirectional transformer pretrained using a combination of masked language modeling objective and next sentence prediction on a large corpus comprising the Toronto Book Corpus and Wikipedia.



In [2]:
import pandas as pd
import numpy as np
import torch
from transformers import BertTokenizer, BertModel
from tensorflow.keras import layers, Model
import tensorflow as tf
from google.colab import drive
drive.mount('/content/drive/')
# Load and preprocess the data
df = pd.read_csv('/content/drive/MyDrive/Projects/Hayat/facebook-fact-check.csv', encoding='latin-1')

# Text preprocessing for BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
max_len = 100

def encode_texts(texts):
    return tokenizer(
        texts.tolist(),
        max_length=max_len,
        padding='max_length',
        truncation=True,
        return_tensors='pt'
    )

text_encodings = encode_texts(df['Context Post'].fillna(''))
input_ids = text_encodings['input_ids']
attention_mask = text_encodings['attention_mask']

# Social features preprocessing
social_features = ['share_count', 'reaction_count', 'comment_count']
social_data = df[social_features].fillna(0).values
social_data = (social_data - social_data.mean()) / (social_data.std() + 1e-7)

# Labels for fake news detection
rating_map = {
    'no factual content': 0,
    'mostly true': 1,
    # Add more ratings based on your dataset
    # For fake news: 0 = fake/not factual, 1 = true/factual
}
labels = df['Rating'].map(rating_map).fillna(0).values

# Convert torch tensors to tf tensors
input_ids_tf = tf.convert_to_tensor(input_ids.numpy())
attention_mask_tf = tf.convert_to_tensor(attention_mask.numpy())
social_data_tf = tf.convert_to_tensor(social_data)
labels_tf = tf.convert_to_tensor(labels)

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [4]:
import pandas as pd
import numpy as np
import torch
from transformers import BertTokenizer, BertModel
from tensorflow.keras import layers, Model
import tensorflow as tf


# Text Branch with BERT - Fixed version
def create_text_branch(bert_model, use_cnn=True):
    input_ids = layers.Input(shape=(max_len,), dtype=tf.int32)
    attention_mask = layers.Input(shape=(max_len,), dtype=tf.int32)

    # Define BERT layer with explicit output shape
    def bert_layer(inputs):
        input_ids, attention_mask = inputs
        with torch.no_grad():
            outputs = bert_model(
                input_ids=torch.tensor(input_ids.numpy()),
                attention_mask=torch.tensor(attention_mask.numpy())
            )
        return outputs.last_hidden_state.numpy()

    # BERT output shape: (batch_size, max_len, 768)
    bert_output = layers.Lambda(
        bert_layer,
        output_shape=(max_len, 768)
    )([input_ids, attention_mask])

    if use_cnn:
        x = layers.Conv1D(64, 5, activation='relu')(bert_output)
        x = layers.MaxPooling1D(5)(x)

    x = layers.Flatten()(x)
    return [input_ids, attention_mask], x

# Social Branch (unchanged)
def create_social_branch(input_shape, use_cnn=True):
    inputs = layers.Input(shape=input_shape)
    x = layers.Dense(64, activation='relu')(inputs)

    if use_cnn:
        x = layers.Reshape((-1, 1))(x)
        x = layers.Conv1D(32, 3, activation='relu')(x)
        x = layers.MaxPooling1D(2)(x)

    x = layers.Flatten()(x)
    return inputs, x

# Complete model (unchanged except for using the fixed text branch)
def create_model(bert_model):
    bert = BertModel.from_pretrained(
        "bert-base-uncased",
        torch_dtype=torch.float16,
        attn_implementation="sdpa"
    )

    text_inputs, text_output = create_text_branch(bert, use_cnn=True)
    social_input, social_output = create_social_branch((len(social_features),), use_cnn=True)

    combined = layers.concatenate([text_output, social_output])
    x = layers.Dense(128, activation='relu')(combined)
    x = layers.Dropout(0.3)(x)
    outputs = layers.Dense(1, activation='sigmoid')(x)

    model = Model(inputs=[text_inputs, social_input], outputs=outputs)
    return model

# [Rest of the code - compilation, training, evaluation - remains the same...]

# Create and compile
bert_model = BertModel.from_pretrained(
    "bert-base-uncased",
    torch_dtype=torch.float16,
    attn_implementation="sdpa"
)
model = create_model(bert_model)
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]
)

model.summary()

# Train
history = model.fit(
    [input_ids_tf, attention_mask_tf, social_data_tf],
    labels_tf,
    epochs=10,
    batch_size=16,
    validation_split=0.2
)

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_6             │ (None, 3)              │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_4             │ (None, 100)            │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_5             │ (None, 100)            │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 64)             │            256 │ input_layer_6[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lambda_2 (Lambda)         │ (None, 100, 768)       │              0 │ input_layer_4[0][0],   │
│                           │                        │                │ input_layer_5[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ reshape (Reshape)         │ (None, 64, 1)          │              0 │ dense[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d (Conv1D)           │ (None, 96, 64)         │        245,824 │ lambda_2[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_1 (Conv1D)         │ (None, 62, 32)         │            128 │ reshape[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling1d             │ (None, 19, 64)         │              0 │ conv1d[0][0]           │
│ (MaxPooling1D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling1d_1           │ (None, 31, 32)         │              0 │ conv1d_1[0][0]         │
│ (MaxPooling1D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ flatten (Flatten)         │ (None, 1216)           │              0 │ max_pooling1d[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ flatten_1 (Flatten)       │ (None, 992)            │              0 │ max_pooling1d_1[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate (Concatenate) │ (None, 2208)           │              0 │ flatten[0][0],         │
│                           │                        │                │ flatten_1[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_1 (Dense)           │ (None, 128)            │        282,752 │ concatenate[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout (Dropout)         │ (None, 128)            │              0 │ dense_1[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_2 (Dense)           │ (None, 1)              │            129 │ dropout[0][0]          │
└──────────────────────

 Total params: 529,089 (2.02 MB)

 Trainable params: 529,089 (2.02 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/models/functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: [['keras_tensor_8', 'keras_tensor_9'], 'keras_tensor_14']
Received: inputs=('Tensor(shape=(None, 100))', 'Tensor(shape=(None, 100))', 'Tensor(shape=(None, 3))')
  warnings.warn(msg)


AttributeError: Exception encountered when calling Lambda.call().

[1m'SymbolicTensor' object has no attribute 'numpy'[0m

Arguments received by Lambda.call():
  • inputs=['tf.Tensor(shape=(None, 100), dtype=int32)', 'tf.Tensor(shape=(None, 100), dtype=int32)']
  • mask=['None', 'None']
  • training=True